# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770795148750                   -0.53    8.0         
  2   -2.772140240627       -2.87       -1.32    1.0    124ms
  3   -2.772170009636       -4.53       -2.52    1.0    108ms
  4   -2.772170687841       -6.17       -3.34    2.0    123ms
  5   -2.772170720601       -7.48       -3.85    1.0    125ms
  6   -2.772170722855       -8.65       -4.95    1.0    112ms
  7   -2.772170723003       -9.83       -4.95    2.0    142ms
  8   -2.772170723015      -10.95       -5.96    1.0    116ms
  9   -2.772170723015      -12.55       -6.78    2.0    148ms
 10   -2.772170723015      -14.05       -6.92    1.0    127ms
 11   -2.772170723015   +  -14.31       -7.37    1.0    125ms
 12   -2.772170723015   +  -13.94       -8.34    2.0    133ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.7

1.7735581622128083

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770749962812                   -0.52    9.0         
  2   -2.772059364926       -2.88       -1.32    1.0    105ms
  3   -2.772082938165       -4.63       -2.43    1.0    118ms
  4   -2.772083328513       -6.41       -3.11    1.0    142ms
  5   -2.772083417616       -7.05       -4.30    2.0    123ms
  6   -2.772083417724       -9.97       -4.54    1.0    129ms
  7   -2.772083417809      -10.07       -5.79    1.0    113ms
  8   -2.772083417811      -11.72       -6.23    2.0    142ms
  9   -2.772083417811   +  -13.45       -6.45    1.0    116ms
 10   -2.772083417811      -13.63       -7.52    2.0    158ms
 11   -2.772083417811      -13.97       -7.72    1.0    117ms
 12   -2.772083417811      -14.27       -8.89    2.0    150ms

Polarizability via ForwardDiff:       1.7725349743197418
Polarizability via finite difference: 1.7735581622128083
